In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import streamlit as st
import joblib
import statsmodels
from PIL import Image, ImageOps


In [9]:
year_to_forecast = 2024
years_into_the_future = year_to_forecast - 2023

In [10]:
#model calculations
model = joblib.load('wind_generation_SARIMA_model.pkl')

In [15]:
#calculate steps for model to forecast from 2023 onwards
#X3_w_test starts May 2020
steps_rem_2020 = 8
steps_2021_to_2023 = 3*12
steps_test = steps_rem_2020 + steps_2021_to_2023 + (12*(int(year_to_forecast)-2023))

#generate forecast
forecast = model.forecast(steps=steps_test)

#calculate total generation for selected year (MWh)
#data is in half-hourly records; divide by 2 to get MWh
selected_yr_forecasted_wind_mwh = forecast.loc[str(year_to_forecast)].sum() / 2

#calculate TWh
selected_yr_forecasted_wind_twh = selected_yr_forecasted_wind_mwh/1000000

#load .csv of non-wind generation in 2023
non_wind_data = pd.read_csv('energy_gen_2023_excl_wind.csv')

#add wind calculations to dataframe
non_wind_data.loc[len(non_wind_data)] = ['WIND', selected_yr_forecasted_mwh]

#calculate total energy generation
total_generation = non_wind_data['Generation (MWh)'].sum()

#calculate total energy opex
total_cost = selected_yr_forecasted_mwh*67
total_cost_bil = total_cost/1000000000

In [22]:
#UK gov March 2024 strike prices for onshore and offshore wind
w_off_strike_price = 73 #per MWh
w_on_strike_price = 64 #per MWh
w_years_to_build = 0.5

#avg build cost per 50-turbine farm
cost_wind_farm = 617380000 #mil GBP per 50-turbine farm, 2.75MW ea

#wind generation per farm = avg 50 turbines * 4380MWh/yr per turbine
w_mwh_year = 219000 

w_total_opex = (w_off_strike_price * selected_yr_forecasted_wind_mwh * 0.66) + (w_on_strike_price * selected_yr_forecasted_wind_mwh * 0.33)
w_total_capex = (years_into_the_future/w_years_to_build)* cost_wind_farm
w_total_farms = selected_yr_forecasted_wind_mwh/w_mwh_year
w_total_cost = ((w_total_capex * w_total_farms) + w_total_opex)/1000000000


w_total_farms = selected_yr_forecasted_wind_mwh/w_mwh_year
w_total_capex = w_total_farms * cost_wind_farm
w_total_opex = (w_off_strike_price * selected_yr_forecasted_wind_mwh * 0.66) + (w_on_strike_price * selected_yr_forecasted_wind_mwh * 0.33)
w_total_cost = w_total_capex + w_total_opex

4.666692286666667